In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import wcopt

import importlib
importlib.reload(wcopt)

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

In [65]:
# Toy model: G fcn and h
G = np.zeros((4,2))
G[0,0] = 1
G[1,0] = 1
G[1,1] = 1
G[1,1] = -1
G[2,1] = 1
G[3,1] = 1
print(G)
p, k   = G.shape
I      = np.identity(2)
lamb   = I[:,1:2]
h      = lambda theta: np.reshape(G @ theta, (p,1))

# Variance
X = np.random.normal(size=(10,p))
V = np.matmul(X.transpose(), X)
print(V)

[[ 1.  0.]
 [ 1. -1.]
 [ 0.  1.]
 [ 0.  1.]]
[[10.81370905  0.86791506  1.07547729  3.72939856]
 [ 0.86791506 16.26419539 -1.61693434 -3.98966489]
 [ 1.07547729 -1.61693434  4.40006733  0.6261296 ]
 [ 3.72939856 -3.98966489  0.6261296  13.79929655]]


In [16]:
# Check worst case optimal computation 
W, stderr, Vout, x, z = wcopt.ComputeWorstCaseOptimal_Single(V, G, lamb, 1e-6)

In [66]:
## Test out the full estimation function and results bundling


# Set parameters for testing
theta_true    = np.reshape(np.array([1,1]), (2,1))
theta0        = np.reshape(np.array([0,0]), (2,1))
cv            = 1.96
l             = -np.inf
u             = np.inf
fullyFeasible = False
Nsim          = 500


muhats        = h(theta_true) + np.random.multivariate_normal(np.zeros(p), V, size=Nsim).T
Gfcn_         = lambda theta: wcopt.ComputeG(h, theta)
Nobs = 1


# Loop over simulations
cov    = np.zeros((5,Nsim,k))
thetas = np.zeros((5,Nsim,k))
for s in range(Nsim):
    
    # Loop over parameters
    for k_ in range(k):
        lamb = I[:,k_]
        res = wcopt.TestSingle(muhats[:,s], V, Nobs, lamb, theta0, theta_true, cv, l, u, fullyFeasible, Gfcn_, h, True)
        cov[:,s,k_] = np.array([res[approach]['coverage'] for approach in ['naive', 'wcopt', 'wcopt_onestep', 'opt', 'opt_onestep']])
        thetas[:,s,k_] = np.array([res[approach]['lcomb'] for approach in ['naive', 'wcopt', 'wcopt_onestep', 'opt', 'opt_onestep']])
        
print(cov[:,:,0].mean(axis=1))
print(cov[:,:,1].mean(axis=1))   

[0.938 0.948 0.948 0.944 0.944]
[0.912 0.948 0.948 0.952 0.952]
